# side model

In [1]:
import numpy as np
import pandas as pd

from strategies.BinanceMLV2.config import SIDE_ALL

df_features = pd.read_parquet("data/features_240m.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(11470, 832)
(11470, 7)


720m_dft_dom_cycle_ddt_lag8    174
720m_adx_14_dt_lag27           171
720m_adx_14_ddt_lag24          165
720m_dft_dom_cycle_ddt_lag1    153
720m_comb_spectrum_pwr_9       150
                              ... 
240m_comb_spectrum_pwr_35        0
240m_comb_spectrum_pwr_27        0
240m_trendflex                   0
240m_chaiken_money_flow          0
240m_norm_on_balance_volume      0
Length: 832, dtype: int64

In [2]:
BAD_FRONT = 174

df_features = df_features[BAD_FRONT:]
label_side = label_side[BAD_FRONT:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(11296, 832)
(11296, 7)


80m_ac_1                   0
80m_ac_29                  0
240m_reactivity_dt_lag3    0
240m_reactivity_lag1       0
240m_reactivity_lag12      0
                          ..
80m_vwap_dt                0
80m_vwap_dt_lag1           0
80m_vwap_dt_lag10          0
80m_vwap_dt_lag2           0
720m_williams_r_lag15      0
Length: 832, dtype: int64

In [7]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(11296, 832)
(array([0, 1]), array([5340, 5956]))
(array([0, 1]), array([1043, 1217]))


In [8]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=150, n_jobs=1)

[I 2025-04-13 13:04:40,973] A new study created in memory with name: no-name-eda85b91-83a2-4733-b63c-3d9a2c1ce0fc
[I 2025-04-13 13:04:44,107] Trial 0 finished with value: 0.8786494618031072 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 263, 'max_depth': 202, 'min_gain_to_split': 0.2982203727127801, 'min_data_in_leaf': 199, 'lambda_l1': 54.82707108061001, 'lambda_l2': 28.65673571775453, 'num_boost_round': 565}. Best is trial 0 with value: 0.8786494618031072.


precision: 0.808455565142364, recall: 0.7699260476581758, f1: 0.7887205387205387


[I 2025-04-13 13:04:50,632] Trial 1 finished with value: 0.8735908915798952 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 241, 'max_depth': 988, 'min_gain_to_split': 0.47641593058065795, 'min_data_in_leaf': 435, 'lambda_l1': 67.89434797363405, 'lambda_l2': 21.97581947969587, 'num_boost_round': 1358}. Best is trial 0 with value: 0.8786494618031072.


precision: 0.8053402239448751, recall: 0.7682826622843056, f1: 0.7863751051303617


[I 2025-04-13 13:04:56,654] Trial 2 finished with value: 0.8703490263768867 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 175, 'max_depth': 116, 'min_gain_to_split': 0.2650229605712233, 'min_data_in_leaf': 77, 'lambda_l1': 88.65505776479515, 'lambda_l2': 41.4754270683249, 'num_boost_round': 1511}. Best is trial 0 with value: 0.8786494618031072.


...

precision: 0.8151487826871056, recall: 0.7428101889893179, f1: 0.7773000859845228


[I 2025-04-13 13:13:29,207] Trial 148 finished with value: 0.881466693872599 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 162, 'max_depth': 800, 'min_gain_to_split': 0.44512612493714826, 'min_data_in_leaf': 414, 'lambda_l1': 17.636417008278606, 'lambda_l2': 58.45133035445933, 'num_boost_round': 387}. Best is trial 118 with value: 0.8841618143730832.


precision: 0.823905109489051, recall: 0.741988496302383, f1: 0.7808041504539559


[I 2025-04-13 13:13:31,857] Trial 149 finished with value: 0.8828564023095631 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 247, 'max_depth': 769, 'min_gain_to_split': 0.3289861611860983, 'min_data_in_leaf': 420, 'lambda_l1': 2.511434095572115, 'lambda_l2': 88.27018896294125, 'num_boost_round': 499}. Best is trial 118 with value: 0.8841618143730832.


precision: 0.8333333333333334, recall: 0.7354149548069022, f1: 0.7813182016586644


In [9]:
study.best_params

{'is_unbalance': True,
 'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 165,
 'max_depth': 918,
 'min_gain_to_split': 0.4576468230010513,
 'min_data_in_leaf': 237,
 'lambda_l1': 12.58983153741093,
 'lambda_l2': 8.939148810557416,
 'num_boost_round': 353}

In [10]:
import lightgbm as lgb  # noqa

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [11]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [12]:
import plotly.express as px
from sklearn.metrics import roc_auc_score  # noqa

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8841618143730832


In [13]:
model.save_model("strategies/BinanceMLV2/model/model_side.txt")
prod_model.save_model("strategies/BinanceMLV2/model/model_side_prod.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from strategies.BinanceMLV2.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_240m.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="strategies/BinanceMLV2/model/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

720m_amihud_lambda_lag3          150
720m_amihud_lambda_lag18         150
720m_amihud_lambda_lag25         150
720m_amihud_lambda_lag29         150
720m_hasbrouck_lambda            150
                                ... 
240m_hasbrouck_lambda_dt           1
240m_amihud_lambda_dt              1
240m_norm_on_balance_volume        0
240m_corwin_schultz_estimator      0
side_model_res                     0
Length: 92, dtype: int64

In [2]:
BAD_FRONT = 150

df_features = df_features[BAD_FRONT:]
meta_label = meta_label[BAD_FRONT:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(11320, 92)
(11320, 7)


80m_ac_20                     0
240m_ma_difference_dt         0
240m_trendflex_dt             0
240m_roll_impact_dt_lag5      0
240m_reactivity_dt            0
                             ..
240m_acc_swing_index_lag15    0
240m_acc_swing_index_lag10    0
240m_acc_swing_index_lag1     0
80m_vwap_lag9                 0
side_model_res                0
Length: 92, dtype: int64

In [3]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(11320, 92)
(array([0, 1]), array([1415, 7641]))
(array([0, 1]), array([ 495, 1769]))


In [4]:
import optuna
from sklearn.metrics import (
    f1_score,  # noqa
    fbeta_score,  # noqa
    precision_score,  # noqa
    recall_score,  # noqa
    roc_auc_score,  # noqa
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.25, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=150, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-13 13:57:48,008] A new study created in memory with name: no-name-98a619f8-6bfb-4f0c-a7a6-dbcca343219c
[I 2025-04-13 13:57:48,899] Trial 0 finished with value: 0.69378111683861 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 370, 'max_depth': 800, 'min_gain_to_split': 0.24005995677667152, 'min_data_in_leaf': 221, 'lambda_l1': 76.43720436061535, 'lambda_l2': 99.67232872043587, 'num_boost_round': 1858}. Best is trial 0 with value: 0.69378111683861.


precision: 0.9235772357723577, recall: 0.6421707179197287, f1: 0.7575858619539847, auc: 0.788832359776396, fbeta: 0.69378111683861


[I 2025-04-13 13:57:53,291] Trial 1 finished with value: 0.738810575942134 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 417, 'max_depth': 120, 'min_gain_to_split': 0.057846076694734294, 'min_data_in_leaf': 26, 'lambda_l1': 5.496063998817878, 'lambda_l2': 6.445538043774487, 'num_boost_round': 1690}. Best is trial 1 with value: 0.738810575942134.


precision: 0.8449704142011835, recall: 0.8072357263990956, f1: 0.8256721595836947, auc: 0.7512696210265458, fbeta: 0.738810575942134


[I 2025-04-13 13:57:53,598] Trial 2 finished with value: 0.6991633469395105 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 367, 'max_depth': 646, 'min_gain_to_split': 0.8857704830038639, 'min_data_in_leaf': 245, 'lambda_l1': 65.04341391397357, 'lambda_l2': 43.50180334102823, 'num_boost_round': 415}. Best is trial 1 with value: 0.738810575942134.


...

precision: 0.8422273781902552, recall: 0.8208027133973996, f1: 0.8313770397938735, auc: 0.7567672199667677, fbeta: 0.7428968038102388


[I 2025-04-13 14:08:31,301] Trial 148 finished with value: 0.7444146757771697 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 481, 'max_depth': 835, 'min_gain_to_split': 0.11456921629377564, 'min_data_in_leaf': 48, 'lambda_l1': 0.2619949405758275, 'lambda_l2': 63.45409919770925, 'num_boost_round': 1349}. Best is trial 67 with value: 0.754667794319041.


precision: 0.8217506631299735, recall: 0.8756359525155455, f1: 0.8478379857690203, auc: 0.757184050796261, fbeta: 0.7444146757771697


[I 2025-04-13 14:08:36,447] Trial 149 finished with value: 0.7420370770639995 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 492, 'max_depth': 854, 'min_gain_to_split': 0.14836672402638076, 'min_data_in_leaf': 31, 'lambda_l1': 6.475488483487725, 'lambda_l2': 71.8109200526309, 'num_boost_round': 1142}. Best is trial 67 with value: 0.754667794319041.


precision: 0.8309232480533927, recall: 0.8445449406444319, f1: 0.8376787216148024, auc: 0.755249498946503, fbeta: 0.7420370770639995


In [8]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 475,
 'max_depth': 767,
 'min_gain_to_split': 0.3119703713225001,
 'min_data_in_leaf': 53,
 'lambda_l1': 1.9958191747928264,
 'lambda_l2': 29.73435625739242,
 'num_boost_round': 1663}

In [9]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [10]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [11]:
model.save_model("strategies/BinanceMLV2/model/model_meta.txt")
prod_model.save_model("strategies/BinanceMLV2/model/model_meta_prod.txt")